In [1]:
import os
import dotenv

In [2]:
import zeep
import json
from tap_tradera.client import TraderaClient

In [3]:
dotenv.load_dotenv(".env")

True

In [4]:
client = TraderaClient(
    app_id=os.getenv("TAP_TRADERA_APP_ID"), app_key=os.getenv("TAP_TRADERA_APP_KEY")
)

Forcing http:address location to HTTPS
Forcing http:address location to HTTPS


In [5]:
search_results = client.search("primus")

In [6]:
search_result = zeep.helpers.serialize_object(search_results[0], dict)
search_result

{'Id': 570065887,
 'ShortDescription': 'Antik fotgen lyckta primus. ',
 'BuyItNowPrice': None,
 'SellerId': 5339175,
 'SellerAlias': 'Lennart234',
 'MaxBid': 1500,
 'ThumbnailLink': 'https://img.tradera.net/thumbs/757/509512757_4566be26-a5e0-43c0-9e62-e68ae20cf810.jpg',
 'SellerDsrAverage': 0.0,
 'EndDate': datetime.datetime(2022, 11, 24, 14, 28, 56, 1000, tzinfo=<FixedOffset '+01:00'>),
 'NextBid': 1500,
 'HasBids': False,
 'IsEnded': False,
 'ItemType': 'Auction',
 'ItemUrl': 'https://www.tradera.com/item/201108/570065887/antik-fotgen-lyckta-primus-',
 'CategoryId': 201108,
 'BidCount': 0,
 'ImageLinks': {'ImageLink': [{'Url': 'https://img.tradera.net/medium/757/509512757_4566be26-a5e0-43c0-9e62-e68ae20cf810.jpg',
    'Format': 'gallery'},
   {'Url': 'https://img.tradera.net/images/757/509512757_4566be26-a5e0-43c0-9e62-e68ae20cf810.jpg',
    'Format': 'normal'},
   {'Url': 'https://img.tradera.net/thumbs/757/509512757_4566be26-a5e0-43c0-9e62-e68ae20cf810.jpg',
    'Format': 'thumb'},

In [15]:
search_items = [
    zeep.helpers.serialize_object(search_item, dict) for search_item in search_results
]
search_items[0]

{'Id': 570065887,
 'ShortDescription': 'Antik fotgen lyckta primus. ',
 'BuyItNowPrice': None,
 'SellerId': 5339175,
 'SellerAlias': 'Lennart234',
 'MaxBid': 1500,
 'ThumbnailLink': 'https://img.tradera.net/thumbs/757/509512757_4566be26-a5e0-43c0-9e62-e68ae20cf810.jpg',
 'SellerDsrAverage': 0.0,
 'EndDate': datetime.datetime(2022, 11, 24, 14, 28, 56, 1000, tzinfo=<FixedOffset '+01:00'>),
 'NextBid': 1500,
 'HasBids': False,
 'IsEnded': False,
 'ItemType': 'Auction',
 'ItemUrl': 'https://www.tradera.com/item/201108/570065887/antik-fotgen-lyckta-primus-',
 'CategoryId': 201108,
 'BidCount': 0,
 'ImageLinks': {'ImageLink': [{'Url': 'https://img.tradera.net/medium/757/509512757_4566be26-a5e0-43c0-9e62-e68ae20cf810.jpg',
    'Format': 'gallery'},
   {'Url': 'https://img.tradera.net/images/757/509512757_4566be26-a5e0-43c0-9e62-e68ae20cf810.jpg',
    'Format': 'normal'},
   {'Url': 'https://img.tradera.net/thumbs/757/509512757_4566be26-a5e0-43c0-9e62-e68ae20cf810.jpg',
    'Format': 'thumb'},

In [20]:
import datetime
from genson import SchemaBuilder
from genson.schema.strategies import TypedSchemaStrategy

class CustomDateTime(TypedSchemaStrategy):
    """
    strategy for date-time formatted strings
    """
    JS_TYPE = 'string'
    PYTHON_TYPE = (str, type(u''))

    # create a new instance variable
    def __init__(self, node_class):
        super().__init__(node_class)
        self.format = "date-time"

    @classmethod
    def match_object(self, obj):
        super().match_object(obj)
        try:
            if isinstance(obj, datetime.datetime):
                return True
            else:
                return False
        except (TypeError, ValueError) as exception:
            #print(exception)
            return False

    def to_schema(self):
        schema = super().to_schema()
        schema['type'] = self.JS_TYPE
        schema['format'] = self.format
        return schema

class CustomSchemaBuilder(SchemaBuilder):
    """ detects & labels date-time formatted strings """
    EXTRA_STRATEGIES = (CustomDateTime, )

In [21]:
builder = CustomSchemaBuilder()
for search_item in search_items:
    builder.add_object(search_item)
json_schema = builder.to_json(indent=2)
print(json_schema)

{
  "$schema": "http://json-schema.org/schema#",
  "type": "object",
  "properties": {
    "Id": {
      "type": "integer"
    },
    "ShortDescription": {
      "type": "string"
    },
    "BuyItNowPrice": {
      "type": [
        "integer",
        "null"
      ]
    },
    "SellerId": {
      "type": "integer"
    },
    "SellerAlias": {
      "type": "string"
    },
    "MaxBid": {
      "type": "integer"
    },
    "ThumbnailLink": {
      "type": "string"
    },
    "SellerDsrAverage": {
      "type": "number"
    },
    "EndDate": {
      "type": "string",
      "format": "date-time"
    },
    "NextBid": {
      "type": "integer"
    },
    "HasBids": {
      "type": "boolean"
    },
    "IsEnded": {
      "type": "boolean"
    },
    "ItemType": {
      "type": "string"
    },
    "ItemUrl": {
      "type": "string"
    },
    "CategoryId": {
      "type": "integer"
    },
    "BidCount": {
      "type": "integer"
    },
    "ImageLinks": {
      "type": "object",
      "prop